<a href="https://colab.research.google.com/github/agishaalbert/NLP_LABS/blob/main/AlbertAgishaLab3n_gram_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI NLP - Review sessions</h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 3: n-gram models </h1>

**Big thanks to Amr Khalifa who improved this lab and made it to a Jupyter Notebook!**

In [12]:
import io, sys, math, re
from collections import defaultdict
import numpy as np

In [13]:
# data_loader
def load_data(filename):
    '''
    parameters:
    filename (string): datafile
    
    Returns:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    '''
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    vocab = defaultdict(lambda:0)
    for line in fin:
        sentence = line.split()
        data.append(sentence)
        for word in sentence:
            vocab[word] += 1
    return data, vocab

In [14]:

print("load training set..")
print("\n")
train_data, vocab = load_data("train1.txt")
print(train_data[0])
print("\n")
print("how :",vocab['how'])
print("load validation set")
valid_data, _ = load_data("valid1.txt")


load training set..


['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']


how : 107
load validation set


In [15]:

# Vocabulaire=zip(list(vocab.keys()),list(vocab.values()))
# for i,j in Vocabulaire:
#   print(i,j)

In [16]:
train_data[0]

['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']

In [17]:
def remove_rare_words(data, vocab, mincount = 1):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    mincount(int): the minimum count 
    
    Returns: 
    data_with_unk(list of lists): data after replacing rare words with <unk> token
    '''
    # replace words in data that are not in the vocab 
    # or have a count that is below mincount
    data_with_unk = []
    ## FILL CODE
    for sentence in data:
        #creating  a list(sentence) that contain our new sentence 
        sentence_w_unk = []
        for word in sentence:
            #if the word exist in our dictionnary
            #we check the count and if the count is greater then 
            #min count we keep otherwise we append the sentence withunk
            if word in vocab and vocab[word]>mincount:
                sentence_w_unk.append(word)
            else:
                sentence_w_unk.append('<unk>')
        data_with_unk.append(sentence_w_unk)
    
    return data_with_unk

In [18]:
print("remove rare words")
train_data = remove_rare_words(train_data, vocab, mincount = 1)
valid_data = remove_rare_words(valid_data, vocab, mincount = 1)
#train_data
print(train_data[0])


remove rare words
['<s>', 'my', '<unk>', "don't", 'speak', '<unk>', '</s>']


In [19]:
def build_ngram(data, n):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    n (int): size of the n-gram
    
    Returns:
    proba (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    
    
    '''
    total_number_words = 0
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))

    for sentence in data:
        sentence = tuple(sentence)#since we cannot have a key of dict being a list
        #we can use a tuple as key
        ## FILL CODE
        # dict can be indexed by tuples
        # store in the same dict all the ngrams
        # by using the context as a key and the word as a value
        for i in range(len(sentence)):
            total_number_words +=1

            for k in range(n):
                # because I have to provide my dictionary
                #with the n grams, n-1 grams, ... 1, Using the backoff
                if i-k < 0:
                    #we break in this case because when we are at the start of a sentence we can't go 
                    #back in history
                    break
                    
                #we index the sentence by the word which is previous to the word i
                #eg. if k is 3, we look at the 3 words before word i
                counts[sentence[i-k:i]][sentence[i]] +=1 
                           

    proba  = defaultdict(lambda: defaultdict(lambda: 0.0))
    # Build the probabilities from the counts
    # Be careful with how you normalize!

    for context in counts.keys():
    ## FILL 
    # The context is the n-gram
        denom =0
        for word in counts[context].keys():
            #for every word we increase the coumt of the denominator
            denom += counts[context][word] #total time the history of the word appears
        for word in counts[context].keys():
            proba[context][word] = counts[context][word]/denom 
    
    return proba

In [20]:
# RUN TO BUILD NGRAM MODEL

n = 2
print("build ngram model with n = ", n)
model = build_ngram(train_data, n)
#model


build ngram model with n =  2


Check model

In [22]:
# for key, value in model.items():
#   print(key,value)

Here, implement a recursive function over shorter and shorter context to compute a "stupid backoff model". An interpolation model can also be implemented this way.

In [23]:
def get_prob(model, context, w):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    context (list of strings): a sentence
    w(string): the word we need to find it's probability given the context
    
    Retunrs:
    prob(float): probability of this word given the context 
    '''

    # code a recursive function over 
    # smaller and smaller context
    # to compute the backoff model
    
    ## FILL CODE
    #stupid backoff(technic to avoid 0 probability) pge 31 lecture 3

    if context in model and w in model[context]:
        return model[context][w]
    else:
        return 0.4*get_prob(model, context[1:], w)

In [24]:
def perplexity(model, data, n):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    data (list of lists): each list is a sentence of the text
    n(int): size of the n-gram
    
    Retunrs:
    prep(float): the preplexity of the model 
    '''
    ## FILL CODE
    perp, T = 0.0, 0 #perplexity and T is the length of the sequence
    for sentence in data:
        sentence = tuple(sentence)#need to use in dict. why tuple
        for i in range(1, len(sentence)):
            nc = min(n-1, i)# number of context
            context = sentence[i-nc:i]
            #perplexity page 50 lecture 3
            perp += -math.log(get_prob(model, context, sentence[i]))
            T += 1
    perp = math.exp(perp/T)
    return perp

In [25]:
import sys

print(sys.getrecursionlimit()) # Prints 1000

1000


In [26]:
# COMPUTE PERPLEXITY ON VALIDATION SET
#we have some of vocabulary world
#when we increase n the perplexity become small
print("The perplexity is", perplexity(model, valid_data, n=n))


The perplexity is 53.582886955857006


In [28]:
# COMPUTE PERPLEXITY ON Train SET
# as we trained the model with train_data it is evident that the perplexity should 
#be smaller
print("The perplexity is", perplexity(model, train_data, n=n))

The perplexity is 21.668651814592515


In [29]:
def get_proba_distrib(model, context):
    ## need to get the the words after the context and their probability of appearance
    ## after this context 
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    context (list of strings): the sentence we need to find the words after it and 
    thier probabilites
    
    Retunrs:
    words_and_probs(dic): {word: probability of word given context}
    
    '''
    # code a recursive function over context
    # to find the longest available ngram
    
    ## FILL CODE
    
    # we are not only retuning the model[context] since this may not have appeared
    if context in model:
        return model[context]
    else:
        return get_proba_distrib(model, context[1:])

In [30]:
def generate(model):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    
    Retunrs:
    sentence (list of strings): a sentence sampled according to the language model. 
    

    '''
    # generate a sentence. A sentence starts with a <s> and ends with a </s>
    # Possiblly a use function is:
    # np.random.choice(x, 1, p = y)

    # where x is a list of things to sample from
    # and y is a list of probability (of the same length as x)
    sentence = ["<s>"]
    while sentence[-1] != "</s>" and len(sentence)<100:
        ## FILL CODE
        proba = get_proba_distrib(model, tuple(sentence))
        #in the list of words we sample a word wrt its proba.
        word = np.random.choice((list(proba.keys())), 1, p = list(proba.values()))
        sentence.append(word[0])
    return sentence

In [33]:
# GENERATE A SENTENCE FROM THE MODEL

print("Generated sentence: ",generate(model))

Generated sentence:  ['<s>', 'this', 'helps', 'his', 'crime.', '</s>']


Once you are done implementing the model, evaluation and generation code, you can try changing the value of `n`, and play with a larger training set (`train2.txt` and `valid2.txt`). You can also try to implement an interpolation model.

In [34]:
print("load training set..")
print("\n")
train_data, vocab = load_data("train2.txt")
print(train_data[0])
print("\n")
print("how :",vocab['how'])
print("load validation set")
valid_data, _ = load_data("valid2.txt")


load training set..


['<s>', 'i', 'liked', 'your', 'idea', 'and', 'adopted', 'it', '.', '</s>']


how : 3195
load validation set


In [35]:
print("remove rare words")
train_data = remove_rare_words(train_data, vocab, mincount = 1)
valid_data = remove_rare_words(valid_data, vocab, mincount = 1)
#train_data
print(train_data[0])


remove rare words
['<s>', 'i', 'liked', 'your', 'idea', 'and', 'adopted', 'it', '.', '</s>']


In [36]:
# RUN TO BUILD NGRAM MODEL

n = 4
print("build ngram model with n = ", n)
model = build_ngram(train_data, n)
#model

build ngram model with n =  4


In [37]:
# COMPUTE PERPLEXITY ON VALIDATION SET

print("The perplexity is", perplexity(model, valid_data, n=n))

The perplexity is 44.617658978200865


In [38]:
# GENERATE A SENTENCE FROM THE MODEL

print("Generated sentence: ",generate(model))

Generated sentence:  ['<s>', 'i', 'will', 'teach', 'you', 'to', 'play', 'the', 'piano', 'after', 'supper', '.', '</s>']
